In [97]:
%reload_ext autoreload
%autoreload 2
from ce_builder import video_stream, watchbox, obj_group, spatialEvent, ce_and, ce_until, ce_follows

In [98]:
# CE 1.1.a
#  Four recce vehicles move toward a bridge

# This creates a group of rec vehicles
rec_group_composition = {"rec_vehicle":4}
recgroup = obj_group(composition=rec_group_composition)
# This gets the sensor named "camera1" (in our case should refer to a video stream id)
bridge_camera = video_stream("camera1")
bridgewatchbox = watchbox(bridge_camera, positions=[(0,0),(1,2)])

se1 = recgroup.enters(bridgewatchbox)

bridgewatchbox.composition(at==0)["rec_vehicle"]==4 and bridgewatchbox.composition(at==1)["rec_vehicle"]!=4
# SEE THE TODO DOC

# Now, we define the function of approaching a bridge
#  Every relation between a group/object/watchbox creates an event
# ce_11a = rec_group.approaches(bridge_watchbox, min_speed="30mph")

# ce_11a.add_constraint(rec_group.max_distance("100m"))
# There's two ways this event is false - either the group composition is wrong, or the 
#   approaching event does not happen.

In [99]:
# CE 1.1.b
#  Rec vehicles split up into groups of two and position themselves on either side of the bridge

# This is above our bridge watchbox
bridgeedgewatchbox1 = watchbox(bridge_camera, positions =[(0,0),(1,1)])
# This is below our bridge watchbox
bridgeedgewatchbox3 = watchbox(bridge_camera, positions =[(0,0),(1,1)])

# A new object group based on rec_group with the additional constraint
#  that the new object group has members from the original rec_group
#  and inherits other constraints (e.g. max distance 100m)
new_rec_composition = {"rec_vehicle":2}
reccepair = obj_group(composition=new_rec_composition)
# recce_pair = rec_group.create_subgroup(num_members=2)

se1 = reccepair.enters(bridgeedgewatchbox1)
se2 = reccepair.enters(bridgeedgewatchbox3)
# Each side of the bridge has a recce_pair
ce11b = ce_and( [ se1, se2 ], \
                   time_constraints=["1s", "200s"])


In [96]:
print(ce11b.states)
print(ce11b.transitions)
ce11b.visualize()

[{'name': 'ce11b-true-', 'children': ['reccepair-present-bridgeedgewatchbox1', 'reccepair-gone-bridgeedgewatchbox1', 'reccepair-present-bridgeedgewatchbox3', 'reccepair-gone-bridgeedgewatchbox3']}, {'name': 'ce11b-false-', 'children': ['reccepair-present-bridgeedgewatchbox1', 'reccepair-gone-bridgeedgewatchbox1', 'reccepair-present-bridgeedgewatchbox3', 'reccepair-gone-bridgeedgewatchbox3']}]
[{'trigger': 'ce11b-and', 'source': 'ce11b-false', 'dest': 'ce11b-true'}, {'trigger': 'ce11b-notand', 'source': 'ce11b-true', 'dest': 'ce11b-false'}, {'trigger': 'reccepair-entered-bridgeedgewatchbox1', 'source': 'reccepair-gone-bridgeedgewatchbox1', 'dest': 'reccepair-present-bridgeedgewatchbox1'}, {'trigger': 'reccepair-exited-bridgeedgewatchbox1', 'source': 'reccepair-present-bridgeedgewatchbox1', 'dest': 'reccepair-gone-bridgeedgewatchbox1'}, {'trigger': 'reccepair-entered-bridgeedgewatchbox3', 'source': 'reccepair-gone-bridgeedgewatchbox3', 'dest': 'reccepair-present-bridgeedgewatchbox3'}, {'

In [5]:
# CE 1.1.c
#  One pair of recce vehicles moves to the side with the tank column

ce_11c = ce_until( [ recce_pair.exits(bridge_edge_watchbox3), \
                   recce_pair.enters(bridge_edge_watchbox1) ],\
                      time_constraints=["0s", "20s"] )

# ce_11c = ce_until( [ recce_pair.exits(bridge_edge_watchbox3), \
#                    recce_pair.enters(bridge_edge_watchbox1) ], \
#                     same_entities=[recce_pair], time_constraints=["0s", "20s"] )

In [6]:
# overall_ce = ce_follows( [ ce_11a, ce_11b, ce_11c ] )
overall_ce = ce_follows( [ ce_11b, ce_11c ] )

 [ ce_11a, ce_11b, ce_11c ] 


In [ ]:
#  Overall CE event

# Basically, this states that each event should follow another.

# Compile the CE into a hierarchical fsm
compiled_ce = overall_ce.compile()
#  Visualize the compiled FSM
compiled_ce.visualize()

In [ ]:
#  Then we can run an experiment
compiled_ce.execute()